In [1]:
import torch
import torchvision
import numpy as np
import logging
import random
from scipy import spatial
from utils import Utils
import os

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")

from benchmark import ImageBenchmark
bench = ImageBenchmark()
models = list(bench.list_models())
models_dict = {}
for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
    print(f'{i}\t {model.__str__()}')
    models_dict[model.__str__()] = model

0	 pretrain(mbnetv2,ImageNet)-
1	 pretrain(resnet18,ImageNet)-
2	 train(mbnetv2,Flower102)-
3	 train(mbnetv2,SDog120)-
4	 train(resnet18,Flower102)-
5	 train(resnet18,SDog120)-
6	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-
7	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.5)-
8	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,1)-
9	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-
10	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-
11	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-
12	 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-
13	 pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-
14	 pretrain(resnet18,ImageNet)-transfer(Flower102,1)-
15	 pretrain(resnet18,ImageNet)-transfer(SDog120,0.1)-
16	 pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-
17	 pretrain(resnet18,ImageNet)-transfer(SDog120,1)-
18	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-quantize(qint8)-
19	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-quantize(float16)-
20	 pretrain(mbnetv2,Im

In [6]:
DEVICE = 'cuda'

def gen_adv_inputs(model, inputs):
    from advertorch.attacks import LinfPGDAttack
    def myloss(yhat, y):
        return -((yhat[:,0]-y[:,0])**2 + 0.1*((yhat[:,1:]-y[:,1:])**2).mean(1)).mean()
        
    model = model.to(DEVICE)
    inputs = torch.from_numpy(inputs).to(DEVICE)
    with torch.no_grad():
        model.eval()
        clean_outputs = model(inputs)
    
    output_shape = clean_outputs.shape
    batch_size = output_shape[0]
    num_classes = output_shape[1]
    
    output_mean = clean_outputs.mean(axis=0)
    target_outputs = output_mean - clean_outputs
    
    y = torch.zeros(size=output_shape).to(DEVICE)
    y[:, :] = 100000
    # more diversity
    y = target_outputs * 1000
#     rand_idx = torch.randint(low=0, high=num_classes, size=(batch_size,))
#     y = torch.nn.functional.one_hot(rand_idx, num_classes=num_classes).to(DEVICE) * 10
#     print(y)
    
    adversary = LinfPGDAttack(
        model, loss_fn=myloss, eps=0.1,
        nb_iter=50, eps_iter=0.01, 
        rand_init=True, clip_min=inputs.min().item(), clip_max=inputs.max().item(),
        targeted=True
    )
    
    adv_inputs = adversary.perturb(inputs, y)
    
    with torch.no_grad():
        model.eval()
        adv_outputs = model(adv_inputs).to('cpu').numpy()
#     print(adv_outputs)
    torch.cuda.empty_cache()
    return adv_inputs.to('cpu').numpy()


model = models_dict['pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-prune(0.2)-']
model.torch_model.to(DEVICE)
seed_inputs = model.get_seed_inputs(100, rand=False)
seed_outputs = model.batch_forward(seed_inputs)
_, seed_preds = seed_outputs.to('cpu').data.max(1)

from datetime import datetime
start_time = datetime.now()
adv_inputs = gen_adv_inputs(model.torch_model, seed_inputs)
adv_outputs = model.batch_forward(adv_inputs)
_, adv_preds = adv_outputs.to('cpu').data.max(1)
model.torch_model.cpu()

time_spent = (datetime.now() - start_time).total_seconds()
print(f'spent {time_spent} seconds')

print(f"seed_preds={seed_preds}, adv_preds={adv_preds}, seed_outputs={seed_outputs}, adv_outputs={adv_outputs}")

spent 18.104814 seconds
seed_preds=tensor([ 92,  20,  14,  80,  23,  85,  17,  50,   1,  37,  19,  30,   8,  65,
         98,  90,  93,   6,  94,  38,   7,  49,  23,  71, 101,  84,  62, 100,
         63,  45,  93,  44,  73, 101,  96,  39,  61,  16,  99,  23,  56,  15,
         53,  78,  72,  13,  98,  64,  20,  10,  35,  61,  14,  70,  42,  32,
         29,  86,   1,  36,  13,  69,   3,  43,  47,  13,  55,  73,  53,   5,
        100,  46,   9,  80,  58,  66,  44,  91,  66,  43,  26,  52,  69,  99,
         98,  45,  11,  34, 100,  23,  59,  48,  89,  99,  96,  83,  72,  43,
         35,  37]), adv_preds=tensor([ 0,  7, 89,  7, 89,  0,  0,  0,  0,  0,  7,  0, 27,  0, 42,  0, 93,  6,
         0,  3,  7,  0,  7,  0, 89, 84, 53,  0,  0, 89, 93, 75, 89, 89,  0,  0,
        83,  0, 99, 89,  0,  0, 53, 78,  0,  0, 98,  7,  7,  0, 89,  0, 89,  7,
        42, 32,  0,  0,  0,  0,  0,  0,  3,  0,  0, 13,  3,  0, 53,  5,  7, 89,
         0, 41,  7,  0, 67, 89, 80,  0, 89, 52,  0,  7,  7, 89,  0,  

In [ ]:
DEVICE = 'cuda'


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    # Background colors:
    GREYBG = '\033[100m'
    REDBG = '\033[101m'
    GREENBG = '\033[102m'
    YELLOWBG = '\033[103m'
    BLUEBG = '\033[104m'
    PINKBG = '\033[105m'
    CYANBG = '\033[106m'


def gen_adv_inputs(model, inputs):
    from advertorch.attacks import LinfPGDAttack
    def myloss(yhat, y):
        return -((yhat[:,0]-y[:,0])**2 + 0.1*((yhat[:,1:]-y[:,1:])**2).mean(1)).mean()
        
    model = model.to(DEVICE)
    inputs = torch.from_numpy(inputs).to(DEVICE)
    with torch.no_grad():
        model.eval()
        clean_outputs = model(inputs)
    
    output_shape = clean_outputs.shape
    batch_size = output_shape[0]
    num_classes = output_shape[1]
    
    output_mean = clean_outputs.mean(axis=0)
    target_outputs = output_mean - clean_outputs
    
#     # No diversity, high divergence
#     y = torch.zeros(size=output_shape).to(DEVICE)
#     y[:, 0] = 100000  # Low diversity, high divergence
#     y[:] = output_mean  # Low diversity, low divergence
    
#     y = target_outputs * 0.1  # High diversity, low divergence 
    y = target_outputs * 1000  # High diversity, high divergence
    
    adversary = LinfPGDAttack(
        model, loss_fn=myloss, eps=1.0,
        nb_iter=50, eps_iter=0.01, 
        rand_init=True, clip_min=inputs.min().item(), clip_max=inputs.max().item(),
        targeted=True
    )
    
    adv_inputs = adversary.perturb(inputs, y)
    
    with torch.no_grad():
        model.eval()
        adv_outputs = model(adv_inputs).to('cpu').numpy()
#     print(adv_outputs)
    torch.cuda.empty_cache()
    return adv_inputs.to('cpu').numpy()

def get_comparable_models(target_model):
    target_model_name = target_model.__str__()
    target_model_segs = target_model_name.split('-')
    parent_model_name = '-'.join(target_model_segs[:-2]) + '-'
    parent_model = models_dict[parent_model_name]
    # print(f'parent_model: {parent_model}')
    reference_models = []
    for model in models:
        if not model.__str__().startswith(target_model_segs[0]):
            reference_models.append(model)
            # print(f'reference_model: {model}')
    return parent_model, reference_models


def compute_ddv(model, normal_inputs, adv_inputs):
    normal_outputs = model.batch_forward(normal_inputs).cpu().numpy()
    adv_outputs = model.batch_forward(adv_inputs).cpu().numpy()
    output_pairs = zip(normal_outputs, adv_outputs)
    # print(list(output_pairs)[0])
    ddv = []  # DDV is short for decision distance vector
    for i, (ya, yb) in enumerate(output_pairs):
        dist = spatial.distance.cosine(ya, yb)
        ddv.append(dist)
    ddv = Utils.normalize(np.array(ddv))
    return ddv


def load_inputs(model):
    inputs_path = os.path.join(model.torch_model_path, 'inputs.npz')
    npzfile = np.load(inputs_path, allow_pickle=True)
    seed_inputs = npzfile['seed_inputs']
    adv_inputs = npzfile['adv_inputs']
    saved_inputs = npzfile['saved_inputs'].item()
    # print(saved_inputs)
    return seed_inputs, adv_inputs
    

for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
    model_name = model.__str__()
    if not model_name.startswith('pretrain'):
        continue
    if len(model_name.split('-')) < 3:
        continue
    if i < 6:
        continue
#     if i != 79:
#         continue
        
#     if 'quantize' not in model_name:
#         continue
#     if not model_name.endswith('prune(0.8)-'):
#         continue
    
    parent_model, ref_models = get_comparable_models(model)
    print(f'{i}\t {model_name} testing')
    
    source_model = model
    if 'quantize' in model_name:
        model.torch_model.cpu()
        source_model = parent_model
    
    seed_inputs = model.get_seed_inputs(100, rand=False)
#     seed_inputs = np.array([seed_inputs[0]] * 100)  # remove diversity of inputs
    adv_inputs = gen_adv_inputs(source_model.torch_model, seed_inputs)
#     seed_inputs, adv_inputs = load_inputs(model)
    source_model.torch_model.to(DEVICE)
    ddv = compute_ddv(model, seed_inputs, adv_inputs)
    # print(f'self_sim: {spatial.distance.cosine(ddv, ddv):.4f}')
    
    parent_sim = 0
    for i, ref_model in enumerate([parent_model] + ref_models):
        if 'quantize' in ref_model.__str__(): # quantized models are equivalent to its teacher model
            continue
        try:
            ref_model.torch_model.to(DEVICE)
            ref_ddv = compute_ddv(ref_model, seed_inputs, adv_inputs)
            ref_sim = spatial.distance.cosine(ddv, ref_ddv)
            ref_model.torch_model.cpu()
            if i == 0:
                parent_sim = ref_sim
                gap = 1
                print(f'parent_sim: {ref_sim:.4f} {ref_model}')
            else:
                gap = ref_sim - parent_sim
                if gap > 0:
                    print(f'ref_sim: {ref_sim:.4f} gap={gap:.4f} {ref_model}')
                else:
                    print(f'{bcolors.WARNING}[ERROR] ref_sim: {ref_sim:.4f} gap={gap:.4f} {ref_model}{bcolors.ENDC}')
        except Exception as e:
            print(f'failed to compare: {ref_model}')
            print(f'exception: {e}')
    # break
    source_model.torch_model.cpu()

6	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)- testing
parent_sim: 0.0340 pretrain(mbnetv2,ImageNet)-
ref_sim: 0.0648 gap=0.0309 pretrain(resnet18,ImageNet)-
ref_sim: 0.1638 gap=0.1298 train(mbnetv2,Flower102)-
ref_sim: 0.1114 gap=0.0774 train(mbnetv2,SDog120)-
ref_sim: 0.1638 gap=0.1298 train(resnet18,Flower102)-
ref_sim: 0.1266 gap=0.0926 train(resnet18,SDog120)-
ref_sim: 0.0964 gap=0.0625 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-
ref_sim: 0.1175 gap=0.0835 pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-
ref_sim: 0.0983 gap=0.0643 pretrain(resnet18,ImageNet)-transfer(Flower102,1)-
ref_sim: 0.0952 gap=0.0612 pretrain(resnet18,ImageNet)-transfer(SDog120,0.1)-
ref_sim: 0.0988 gap=0.0648 pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-
ref_sim: 0.0995 gap=0.0656 pretrain(resnet18,ImageNet)-transfer(SDog120,1)-
ref_sim: 0.0908 gap=0.0568 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-prune(0.2)-
ref_sim: 0.0913 gap=0.0573 pretrain(resnet18,ImageNet)-transfer

ref_sim: 0.0559 gap=0.0190 pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-distill()-
ref_sim: 0.0573 gap=0.0203 pretrain(resnet18,ImageNet)-transfer(Flower102,1)-distill()-
ref_sim: 0.1312 gap=0.0942 pretrain(resnet18,ImageNet)-transfer(SDog120,0.1)-distill()-
ref_sim: 0.1399 gap=0.1029 pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-distill()-
ref_sim: 0.0910 gap=0.0540 pretrain(resnet18,ImageNet)-transfer(SDog120,1)-distill()-
ref_sim: 0.1161 gap=0.0791 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-steal(mbnetv2)-
ref_sim: 0.1856 gap=0.1486 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-steal(resnet18)-
ref_sim: 0.1220 gap=0.0851 pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-steal(mbnetv2)-
ref_sim: 0.2164 gap=0.1795 pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-steal(resnet18)-
ref_sim: 0.1374 gap=0.1004 pretrain(resnet18,ImageNet)-transfer(Flower102,1)-steal(mbnetv2)-
ref_sim: 0.1949 gap=0.1579 pretrain(resnet18,ImageNet)-transfer(Flower102,1)-steal(re

ref_sim: 0.2097 gap=0.1618 train(mbnetv2,Flower102)-steal(mbnetv2)-
ref_sim: 0.2561 gap=0.2083 train(mbnetv2,Flower102)-steal(resnet18)-
ref_sim: 0.1457 gap=0.0979 train(mbnetv2,SDog120)-steal(mbnetv2)-
ref_sim: 0.1505 gap=0.1027 train(mbnetv2,SDog120)-steal(resnet18)-
ref_sim: 0.1850 gap=0.1371 train(resnet18,Flower102)-steal(mbnetv2)-
ref_sim: 0.2578 gap=0.2100 train(resnet18,Flower102)-steal(resnet18)-
ref_sim: 0.1477 gap=0.0998 train(resnet18,SDog120)-steal(mbnetv2)-
ref_sim: 0.1402 gap=0.0923 train(resnet18,SDog120)-steal(resnet18)-
9	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)- testing
parent_sim: 0.0199 pretrain(mbnetv2,ImageNet)-
ref_sim: 0.0706 gap=0.0507 pretrain(resnet18,ImageNet)-
ref_sim: 0.1271 gap=0.1072 train(mbnetv2,Flower102)-
ref_sim: 0.0783 gap=0.0584 train(mbnetv2,SDog120)-
ref_sim: 0.0926 gap=0.0727 train(resnet18,Flower102)-
ref_sim: 0.0951 gap=0.0752 train(resnet18,SDog120)-
ref_sim: 0.1100 gap=0.0901 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-
re

ref_sim: 0.0939 gap=0.0601 pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-prune(0.5)-
ref_sim: 0.1119 gap=0.0781 pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-prune(0.8)-
ref_sim: 0.0757 gap=0.0419 pretrain(resnet18,ImageNet)-transfer(SDog120,1)-prune(0.2)-
ref_sim: 0.0792 gap=0.0454 pretrain(resnet18,ImageNet)-transfer(SDog120,1)-prune(0.5)-
ref_sim: 0.0948 gap=0.0610 pretrain(resnet18,ImageNet)-transfer(SDog120,1)-prune(0.8)-
ref_sim: 0.0861 gap=0.0523 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-distill()-
ref_sim: 0.0717 gap=0.0379 pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-distill()-
ref_sim: 0.0715 gap=0.0377 pretrain(resnet18,ImageNet)-transfer(Flower102,1)-distill()-
ref_sim: 0.1211 gap=0.0873 pretrain(resnet18,ImageNet)-transfer(SDog120,0.1)-distill()-
ref_sim: 0.1133 gap=0.0795 pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-distill()-
ref_sim: 0.1072 gap=0.0734 pretrain(resnet18,ImageNet)-transfer(SDog120,1)-distill()-
ref_sim: 0.1208 gap=0.0870 pret

ref_sim: 0.1148 gap=0.0751 train(resnet18,Flower102)-prune(0.5)-
ref_sim: 0.1161 gap=0.0764 train(resnet18,Flower102)-prune(0.8)-
ref_sim: 0.0813 gap=0.0416 train(resnet18,SDog120)-prune(0.2)-
ref_sim: 0.0818 gap=0.0420 train(resnet18,SDog120)-prune(0.5)-
ref_sim: 0.0838 gap=0.0440 train(resnet18,SDog120)-prune(0.8)-
ref_sim: 0.1499 gap=0.1101 train(mbnetv2,Flower102)-distill()-
ref_sim: 0.1334 gap=0.0937 train(mbnetv2,SDog120)-distill()-
ref_sim: 0.1237 gap=0.0839 train(resnet18,Flower102)-distill()-
ref_sim: 0.1255 gap=0.0857 train(resnet18,SDog120)-distill()-
ref_sim: 0.1641 gap=0.1244 train(mbnetv2,Flower102)-steal(mbnetv2)-
ref_sim: 0.2011 gap=0.1614 train(mbnetv2,Flower102)-steal(resnet18)-
ref_sim: 0.1552 gap=0.1154 train(mbnetv2,SDog120)-steal(mbnetv2)-
ref_sim: 0.2016 gap=0.1618 train(mbnetv2,SDog120)-steal(resnet18)-
ref_sim: 0.1401 gap=0.1004 train(resnet18,Flower102)-steal(mbnetv2)-
ref_sim: 0.2030 gap=0.1633 train(resnet18,Flower102)-steal(resnet18)-
ref_sim: 0.1323 gap=0.

...

ref_sim: 0.1151 gap=0.1047 train(resnet18,SDog120)-distill()-
ref_sim: 0.1928 gap=0.1824 train(mbnetv2,Flower102)-steal(mbnetv2)-
ref_sim: 0.2562 gap=0.2457 train(mbnetv2,Flower102)-steal(resnet18)-
ref_sim: 0.1361 gap=0.1257 train(mbnetv2,SDog120)-steal(mbnetv2)-
ref_sim: 0.1793 gap=0.1689 train(mbnetv2,SDog120)-steal(resnet18)-
ref_sim: 0.2128 gap=0.2023 train(resnet18,Flower102)-steal(mbnetv2)-
ref_sim: 0.2581 gap=0.2477 train(resnet18,Flower102)-steal(resnet18)-
ref_sim: 0.1415 gap=0.1310 train(resnet18,SDog120)-steal(mbnetv2)-
ref_sim: 0.1138 gap=0.1033 train(resnet18,SDog120)-steal(resnet18)-
61	 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-prune(0.5)- testing
parent_sim: 0.0099 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-
ref_sim: 0.0544 gap=0.0445 pretrain(mbnetv2,ImageNet)-
ref_sim: 0.1341 gap=0.1242 train(mbnetv2,Flower102)-
ref_sim: 0.0860 gap=0.0760 train(mbnetv2,SDog120)-
ref_sim: 0.1140 gap=0.1040 train(resnet18,Flower102)-
ref_sim: 0.0909 gap=0.0809 train(

ref_sim: 0.0759 gap=0.0494 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-prune(0.2)-
ref_sim: 0.0811 gap=0.0546 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-prune(0.5)-
ref_sim: 0.1669 gap=0.1405 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-prune(0.8)-
ref_sim: 0.0844 gap=0.0579 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-prune(0.2)-
ref_sim: 0.0884 gap=0.0619 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-prune(0.5)-
ref_sim: 0.1311 gap=0.1046 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-prune(0.8)-
ref_sim: 0.0684 gap=0.0420 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-distill()-
ref_sim: 0.0569 gap=0.0304 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.5)-distill()-
ref_sim: 0.0677 gap=0.0412 pretrain(mbnetv2,ImageNet)-transfer(Flower102,1)-distill()-
ref_sim: 0.1428 gap=0.1163 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-distill()-
ref_sim: 0.0858 gap=0.0594 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-distill()-
ref_sim: 0.1114 gap=0.0849 pretrain(mbn

ref_sim: 0.1089 gap=0.1067 train(resnet18,Flower102)-prune(0.2)-
ref_sim: 0.1121 gap=0.1098 train(resnet18,Flower102)-prune(0.5)-
ref_sim: 0.1138 gap=0.1116 train(resnet18,Flower102)-prune(0.8)-
ref_sim: 0.1024 gap=0.1002 train(resnet18,SDog120)-prune(0.2)-
ref_sim: 0.1000 gap=0.0977 train(resnet18,SDog120)-prune(0.5)-
ref_sim: 0.0983 gap=0.0960 train(resnet18,SDog120)-prune(0.8)-
ref_sim: 0.1797 gap=0.1774 train(mbnetv2,Flower102)-distill()-
ref_sim: 0.1722 gap=0.1699 train(mbnetv2,SDog120)-distill()-
ref_sim: 0.1530 gap=0.1507 train(resnet18,Flower102)-distill()-
ref_sim: 0.1336 gap=0.1313 train(resnet18,SDog120)-distill()-
ref_sim: 0.2028 gap=0.2005 train(mbnetv2,Flower102)-steal(mbnetv2)-
ref_sim: 0.2446 gap=0.2423 train(mbnetv2,Flower102)-steal(resnet18)-
ref_sim: 0.1925 gap=0.1903 train(mbnetv2,SDog120)-steal(mbnetv2)-
ref_sim: 0.2118 gap=0.2096 train(mbnetv2,SDog120)-steal(resnet18)-
ref_sim: 0.1823 gap=0.1801 train(resnet18,Flower102)-steal(mbnetv2)-
ref_sim: 0.2471 gap=0.2448 

ref_sim: 0.0685 gap=0.0553 pretrain(mbnetv2,ImageNet)-transfer(Flower102,1)-prune(0.2)-
ref_sim: 0.0624 gap=0.0492 pretrain(mbnetv2,ImageNet)-transfer(Flower102,1)-prune(0.5)-
ref_sim: 0.0976 gap=0.0844 pretrain(mbnetv2,ImageNet)-transfer(Flower102,1)-prune(0.8)-
ref_sim: 0.0679 gap=0.0548 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-prune(0.2)-
ref_sim: 0.0764 gap=0.0633 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-prune(0.5)-
ref_sim: 0.1207 gap=0.1076 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-prune(0.8)-
ref_sim: 0.0588 gap=0.0457 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-prune(0.2)-
ref_sim: 0.0645 gap=0.0514 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-prune(0.5)-
ref_sim: 0.1547 gap=0.1416 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-prune(0.8)-
ref_sim: 0.0641 gap=0.0510 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-prune(0.2)-
ref_sim: 0.0661 gap=0.0530 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-prune(0.5)-
ref_sim: 0.1311 gap=0.1180 pretrain(

ref_sim: 0.1076 gap=0.1060 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-steal(resnet18)-
ref_sim: 0.1380 gap=0.1364 train(mbnetv2,Flower102)-prune(0.2)-
ref_sim: 0.1416 gap=0.1401 train(mbnetv2,Flower102)-prune(0.5)-
ref_sim: 0.1113 gap=0.1097 train(mbnetv2,Flower102)-prune(0.8)-
ref_sim: 0.0784 gap=0.0768 train(mbnetv2,SDog120)-prune(0.2)-
ref_sim: 0.0965 gap=0.0949 train(mbnetv2,SDog120)-prune(0.5)-
ref_sim: 0.0863 gap=0.0847 train(mbnetv2,SDog120)-prune(0.8)-
ref_sim: 0.1000 gap=0.0985 train(resnet18,Flower102)-prune(0.2)-
ref_sim: 0.1022 gap=0.1007 train(resnet18,Flower102)-prune(0.5)-
ref_sim: 0.0999 gap=0.0984 train(resnet18,Flower102)-prune(0.8)-
ref_sim: 0.0748 gap=0.0732 train(resnet18,SDog120)-prune(0.2)-
ref_sim: 0.0736 gap=0.0720 train(resnet18,SDog120)-prune(0.5)-
ref_sim: 0.0717 gap=0.0702 train(resnet18,SDog120)-prune(0.8)-
ref_sim: 0.1656 gap=0.1641 train(mbnetv2,Flower102)-distill()-
ref_sim: 0.1104 gap=0.1088 train(mbnetv2,SDog120)-distill()-
ref_sim: 0.1316 gap=0.1

In [ ]:
x = np.random.randn(10, 3, 3)
y = np.array([x[0]] * 10)
print(x, y)